In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df1 = pd.read_csv('../input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv')
df1.head()

In [ ]:
df1['salary'].isnull().sum()

In [ ]:
df1[df1.status=='Not Placed'].shape

This Means the data is Logically Fine

In [ ]:
df2 = df1.drop(['salary', 'sl_no'], axis=1)
df2.shape

In [ ]:
df2.dtypes

In [ ]:
import seaborn as sns
sns.pairplot(df2)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df2['gender'] = le.fit_transform(df2['gender'])
df2['ssc_b'] = le.fit_transform(df2['ssc_b'])
df2['hsc_b'] = le.fit_transform(df2['hsc_b'])
df2['hsc_s'] = le.fit_transform(df2['hsc_s'])
df2['degree_t'] = le.fit_transform(df2['degree_t'])
df2['workex'] = le.fit_transform(df2['workex'])
df2['specialisation'] = le.fit_transform(df2['specialisation'])
df2['status'] = le.fit_transform(df2['status'])

In [ ]:
df2.head()

In [ ]:
df2.shape

In [ ]:
df2.dtypes

In [ ]:
x = df2.iloc[:,:12]
y = df2['status']
x.shape, y.shape

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(x,y)

In [ ]:
print(model.)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
feat_importances = pd.Series(model.feature_importances_, index=x.columns)
feat_importances.nlargest(5).plot(kind='barh')
plt.show()

Thus Let us take the best 5 Features

In [ ]:
X = x[['etest_p', 'mba_p', 'hsc_p', 'degree_p', 'ssc_p']]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

Hyper Parameter Tuning using OPTUNA

In [ ]:
import optuna
import sklearn.svm
def objective(trial):

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 200, 2000,10)
        max_depth = int(trial.suggest_float('max_depth', 10, 100, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        
        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf,x_train,y_train, n_jobs=-1, cv=3).mean()

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
trial

In [ ]:
study.best_params

In [ ]:
rf=sklearn.ensemble.RandomForestClassifier(n_estimators=490,max_depth=91.71293127875516)
rf.fit(x_train,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
y_pred=rf.predict(x_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
optuna.visualization.plot_contour(study, params=['n_estimators', 'max_depth'])